In [6]:
import requests
import pandas as pd

url="https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20200516&stockNo=2330&_=1589583929315"

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

res=requests.get(url, headers)

if res.status_code == requests.codes.ok:
    print("OK")

res.json()

OK


{'stat': 'OK',
 'date': '20200516',
 'title': '109年05月 2330 台積電           各日成交資訊',
 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'],
 'data': [['109/05/04',
   '72,252,861',
   '21,341,931,218',
   '294.50',
   '296.50',
   '294.00',
   '295.00',
   '-9.50',
   '37,736'],
  ['109/05/05',
   '23,988,405',
   '7,104,774,785',
   '296.50',
   '298.00',
   '295.00',
   '295.50',
   '+0.50',
   '13,569'],
  ['109/05/06',
   '35,704,479',
   '10,524,838,077',
   '294.50',
   '296.00',
   '292.50',
   '296.00',
   '+0.50',
   '16,046'],
  ['109/05/07',
   '28,017,198',
   '8,321,170,642',
   '294.50',
   '299.00',
   '294.50',
   '297.50',
   '+1.50',
   '13,149'],
  ['109/05/08',
   '35,613,893',
   '10,619,091,212',
   '300.00',
   '300.00',
   '296.00',
   '297.50',
   ' 0.00',
   '14,345'],
  ['109/05/11',
   '32,379,019',
   '9,729,431,840',
   '300.00',
   '301.50',
   '298.50',
   '301.00',
   '+3.50',
   '14,920'],
  ['109/05/12',
   '52,494,788',
   '15,5

In [6]:
import datetime
import urllib.parse
from collections import namedtuple
import time
import random
#from twstock.proxy import get_proxies

try:
    from json.decoder import JSONDecodeError
except ImportError:
    JSONDecodeError = ValueError

import requests
from codes import codes


"""
try:
    from . import analytics
    from .codes import codes
except ImportError as e:
    if e.name == 'lxml':
        # Fix #69
        raise e
    import analytics
    from codes import codes
"""

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}


TWSE_BASE_URL = 'http://www.twse.com.tw/'
TPEX_BASE_URL = 'http://www.tpex.org.tw/'
DATATUPLE = namedtuple('Data', ['date', 'capacity', 'turnover', 'open',
                                'high', 'low', 'close', 'change', 'transaction'])


class BaseFetcher(object):
    def fetch(self, year, month, sid, retry):
        pass

    def _convert_date(self, date):
        """Convert '106/05/01' to '2017/05/01'"""
        return '/'.join([str(int(date.split('/')[0]) + 1911)] + date.split('/')[1:])

    def _make_datatuple(self, data):
        pass

    def purify(self, original_data):
        pass


class TWSEFetcher(BaseFetcher):
    REPORT_URL = urllib.parse.urljoin(
        TWSE_BASE_URL, 'exchangeReport/STOCK_DAY')

    def __init__(self):
        pass

    def fetch(self, year: int, month: int, sid: str, retry: int=5):
        params = {'date': '%d%02d01' % (year, month), 'stockNo': sid}
        for retry_i in range(retry):
            r = requests.get(self.REPORT_URL, params=params, headers=headers)
            try:
                data = r.json()
            except JSONDecodeError:
                continue
            else:
                break
        else:
            # Fail in all retries
            data = {'stat': '', 'data': []}

        if data['stat'] == 'OK':
            data['data'] = self.purify(data)
        else:
            data['data'] = []
        time.sleep(random.uniform(2,5))
        return data

    def _make_datatuple(self, data):
        data[0] = datetime.datetime.strptime(
            self._convert_date(data[0]), '%Y/%m/%d')
        data[1] = int(data[1].replace(',', ''))
        data[2] = int(data[2].replace(',', ''))
        data[3] = None if data[3] == '--' else float(data[3].replace(',', ''))
        data[4] = None if data[4] == '--' else float(data[4].replace(',', ''))
        data[5] = None if data[5] == '--' else float(data[5].replace(',', ''))
        data[6] = None if data[6] == '--' else float(data[6].replace(',', ''))
        # +/-/X表示漲/跌/不比價
        data[7] = float(0.0 if data[7].replace(',', '') ==
                        'X0.00' else data[7].replace(',', ''))
        data[8] = int(data[8].replace(',', ''))
        return DATATUPLE(*data)

    def purify(self, original_data):
        return [self._make_datatuple(d) for d in original_data['data']]


class TPEXFetcher(BaseFetcher):
    REPORT_URL = urllib.parse.urljoin(TPEX_BASE_URL,
                                      'web/stock/aftertrading/daily_trading_info/st43_result.php')

    def __init__(self):
        pass

    def fetch(self, year: int, month: int, sid: str, retry: int=5):
        params = {'d': '%d/%d' % (year - 1911, month), 'stkno': sid}
        for retry_i in range(retry):
            r = requests.get(self.REPORT_URL, params=params,
                             proxies=get_proxies())
            try:
                data = r.json()
            except JSONDecodeError:
                continue
            else:
                break
        else:
            # Fail in all retries
            data = {'aaData': []}

        data['data'] = []
        if data['aaData']:
            data['data'] = self.purify(data)
        
        return data

    def _convert_date(self, date):
        """Convert '106/05/01' to '2017/05/01'"""
        return '/'.join([str(int(date.split('/')[0]) + 1911)] + date.split('/')[1:])

    def _make_datatuple(self, data):
        data[0] = datetime.datetime.strptime(self._convert_date(data[0].replace('＊', '')),
                                             '%Y/%m/%d')
        data[1] = int(data[1].replace(',', '')) * 1000
        data[2] = int(data[2].replace(',', '')) * 1000
        data[3] = None if data[3] == '--' else float(data[3].replace(',', ''))
        data[4] = None if data[4] == '--' else float(data[4].replace(',', ''))
        data[5] = None if data[5] == '--' else float(data[5].replace(',', ''))
        data[6] = None if data[6] == '--' else float(data[6].replace(',', ''))
        data[7] = float(data[7].replace(',', ''))
        data[8] = int(data[8].replace(',', ''))
        return DATATUPLE(*data)

    def purify(self, original_data):
        return [self._make_datatuple(d) for d in original_data['aaData']]


class Stock():

    def __init__(self, sid: str, initial_fetch: bool=True):
        self.sid = sid
        self.fetcher = TWSEFetcher(
        ) if codes[sid].market == '上市' else TPEXFetcher()
        self.raw_data = []
        self.data = []

        # Init data
        if initial_fetch:
            self.fetch_31()

    def _month_year_iter(self, start_month, start_year, end_month, end_year):
        ym_start = 12 * start_year + start_month - 1
        ym_end = 12 * end_year + end_month
        for ym in range(ym_start, ym_end):
            y, m = divmod(ym, 12)
            yield y, m + 1

    def fetch(self, year: int, month: int):
        """Fetch year month data"""
        self.raw_data = [self.fetcher.fetch(year, month, self.sid)]
        self.data = self.raw_data[0]['data']
        return self.data

    def fetch_from(self, year: int, month: int):
        """Fetch data from year, month to current year month data"""
        self.raw_data = []
        self.data = []
        today = datetime.datetime.today()
        for year, month in self._month_year_iter(month, year, today.month, today.year):
            self.raw_data.append(self.fetcher.fetch(year, month, self.sid))
            self.data.extend(self.raw_data[-1]['data'])
        return self.data

    def fetch_31(self):
        """Fetch 31 days data"""
        today = datetime.datetime.today()
        before = today - datetime.timedelta(days=60)
        self.fetch_from(before.year, before.month)
        self.data = self.data[-31:]
        return self.data

    @property
    def date(self):
        return [d.date for d in self.data]

    @property
    def capacity(self):
        return [d.capacity for d in self.data]

    @property
    def turnover(self):
        return [d.turnover for d in self.data]

    @property
    def price(self):
        return [d.close for d in self.data]

    @property
    def high(self):
        return [d.high for d in self.data]

    @property
    def low(self):
        return [d.low for d in self.data]

    @property
    def open(self):
        return [d.open for d in self.data]

    @property
    def close(self):
        return [d.close for d in self.data]

    @property
    def change(self):
        return [d.change for d in self.data]

    @property
    def transaction(self):
        return [d.transaction for d in self.data]

In [9]:
stock=Stock('2330')
stock.fetch_31
print(stock.data)
#stock.fetch_from(2020, 1)

NameError: name 'data' is not defined

[Data(date=datetime.datetime(2020, 3, 31, 0, 0), capacity=53901923, turnover=14712557472, open=273.0, high=274.0, low=269.5, close=274.0, change=6.5, transaction=20343),
 Data(date=datetime.datetime(2020, 4, 1, 0, 0), capacity=47572034, turnover=13022612818, open=276.5, high=276.5, low=271.5, close=271.5, change=-2.5, transaction=19580),
 Data(date=datetime.datetime(2020, 4, 6, 0, 0), capacity=59712754, turnover=16324198154, open=273.0, high=275.5, low=270.0, close=275.5, change=4.0, transaction=19971),
 Data(date=datetime.datetime(2020, 4, 7, 0, 0), capacity=48887346, turnover=13817936851, open=283.5, high=284.0, low=280.5, close=283.0, change=7.5, transaction=24281),
 Data(date=datetime.datetime(2020, 4, 8, 0, 0), capacity=38698826, turnover=11016972354, open=285.0, high=285.5, low=283.0, close=285.0, change=2.0, transaction=19126),
 Data(date=datetime.datetime(2020, 4, 9, 0, 0), capacity=29276430, turnover=8346209654, open=287.5, high=288.0, low=282.5, close=283.0, change=-2.0, tran